<h1> Allocation Service - Notebook 3 </h1>
This notebook demonstrates a real world use case of the Allocation Service. We will create two orders for the same instrument for separate portfolios, along with the supporting entities. These orders will be placed with two different brokers. We will see how the Allocation Service handles executions for both of these placements.

<h2> Imports </h2>
Import the required packages for this example.

In [1]:
import lusid
import lusid.api as la
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
)
from helpers import AllocationServiceHelpers as helpers

import pandas as pd
import os

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.6.11172.0


<h2> Setup Constants </h2>
We set up everything in the same scope for simplicity.

In [2]:
scope = "AllocationService_Example3"

<h2> Upsert Instrument </h2>
We use a simple AMZN equity for this example.

In [3]:
instr_df = pd.read_csv("data/3/instruments.csv")
display(instr_df)

,instrument_name,client_internal,currency,isin,figi,ticker
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,AMZN


In [4]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "client_internal",
        "Isin": "isin",
        "Figi": "figi",
    },
    "required": {
        "name": "instrument_name"
    },
}
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


<h2> Upsert Portfolios </h2>
We have two different portfolios, which will have their own respective orders.

In [5]:
pf_df = pd.read_csv("data/3/portfolios.csv")
display(pf_df)

,portfolio_name,portfolio_code,currency,created_date
0,Allocation Service - Example 2 - Portfolio A,PORT001,USD,2023-04-10T12:00:00+00:00
1,Allocation Service - Example 2 - Portfolio B,PORT002,USD,2023-04-10T12:00:00+00:00


In [6]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "currency",
    },
    "optional": {
        "created": "created_date"
    }
}
result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional={},
        file_type="portfolios",
    )

succ, errors = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "errors": len(errors)}])

,success,errors
0,2,0


<h2> Upsert Orders </h2>
Here we have two orders for AMZN stock, both for ten units each with Order 2 being placed 5 minutes before Order 1.

In [7]:
ord_df = pd.read_csv("data/3/orders.csv")
display(ord_df)

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,order_id,side,type,state,date
0,PORT001,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,10,1550,USD,ORD001,buy,limit,new,2023-04-11T11:10:00+00:00
1,PORT002,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,10,1550,USD,ORD002,buy,limit,new,2023-04-11T11:15:00+00:00


In [8]:
set_request = helpers.create_order_set_request_from_df(ord_df, scope)
response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
    order_set_request=set_request)

display(lusid_response_to_data_frame(response))

,properties,version.effective_from,version.as_at_date,version.as_at_created,version.user_id_created,version.as_at_modified,version.user_id_modified,version.as_at_version_number,instrument_identifiers.Instrument/default/Figi,quantity,side,portfolio_id.scope,portfolio_id.code,id.scope,id.code,instrument_scope,lusid_instrument_id,state,type,date,price.amount,price.currency
0,{},0001-01-01 00:00:00+00:00,2023-04-21 15:51:48.119564+00:00,2023-04-20 13:41:10.645795+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:51:48.119564+00:00,00ucmrzqsbhjXJHmz2p7,8,BBG000BVPXP1,10.0,buy,AllocationService_Example3,PORT001,AllocationService_Example3,ORD001,default,LUID_0000MIOX,new,limit,2023-04-11 11:10:00+00:00,10.0,USD
1,{},0001-01-01 00:00:00+00:00,2023-04-21 15:51:48.119564+00:00,2023-04-20 13:41:10.645795+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:51:48.119564+00:00,00ucmrzqsbhjXJHmz2p7,8,BBG000BVPXP1,10.0,buy,AllocationService_Example3,PORT002,AllocationService_Example3,ORD002,default,LUID_0000MIOX,new,limit,2023-04-11 11:15:00+00:00,10.0,USD


<h2> Upsert Block </h2>
The Block contains both of our orders.

In [9]:
block_df = pd.read_csv("data/3/blocks.csv")
display(block_df)

,block_id,order_ids,instrument_name,client_internal,isin,figi,quantity,side,type,time_in_force,created_date
0,BLOCK001,"ORD001,ORD002",Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,20,buy,limit,GTC,2023-04-11T11:15:00+00:00


In [10]:
set_request = helpers.create_block_set_request_from_df(block_df, scope)
response = api_factory.build(lusid.api.BlocksApi).upsert_blocks(
    block_set_request=set_request)

display(lusid_response_to_data_frame(response))

,id.scope,id.code,order_ids.0.scope,order_ids.0.code,order_ids.1.scope,order_ids.1.code,properties,instrument_identifiers.Instrument/default/Figi,lusid_instrument_id,quantity,side,type,time_in_force,created_date,version.effective_from,version.as_at_date,version.as_at_created,version.user_id_created,version.as_at_modified,version.user_id_modified,version.as_at_version_number
0,AllocationService_Example3,BLOCK001,AllocationService_Example3,ORD001,AllocationService_Example3,ORD002,{},BBG000BVPXP1,LUID_0000MIOX,20.0,buy,limit,GTC,2023-04-11 11:15:00+00:00,0001-01-01 00:00:00+00:00,2023-04-21 15:51:48.348465+00:00,2023-04-20 13:41:10.808179+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:51:48.348465+00:00,00ucmrzqsbhjXJHmz2p7,8


<h2> Upsert Placements </h2>
Both placements reference our Block.

In [11]:
placement_df = pd.read_csv("data/3/placements.csv")
display(placement_df)

,placement_id,block_id,instrument_name,client_internal,isin,figi,quantity,state,side,type,time_in_force,created_date
0,PLC001,BLOCK001,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,10,some_state,buy,limit,GTC,2023-04-11T11:15:00+00:00
1,PLC002,BLOCK001,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,10,some_state,buy,limit,GTC,2023-04-11T11:15:00+00:00


In [12]:
set_request = helpers.create_placement_set_request_from_df(placement_df, scope)
response = api_factory.build(lusid.api.PlacementsApi).upsert_placements(
    placement_set_request=set_request)

display(lusid_response_to_data_frame(response))

,id.scope,id.code,block_ids.0.scope,block_ids.0.code,properties,instrument_identifiers.Instrument/default/Figi,lusid_instrument_id,quantity,state,side,time_in_force,type,created_date,version.effective_from,version.as_at_date,version.as_at_created,version.user_id_created,version.as_at_modified,version.user_id_modified,version.as_at_version_number
0,AllocationService_Example3,PLC002,AllocationService_Example3,BLOCK001,{},BBG000BVPXP1,LUID_0000MIOX,10.0,some_state,buy,GTC,limit,2023-04-11 11:15:00+00:00,0001-01-01 00:00:00+00:00,2023-04-21 15:51:48.524844+00:00,2023-04-20 13:58:45.616682+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:51:48.524844+00:00,00ucmrzqsbhjXJHmz2p7,7
1,AllocationService_Example3,PLC001,AllocationService_Example3,BLOCK001,{},BBG000BVPXP1,LUID_0000MIOX,10.0,some_state,buy,GTC,limit,2023-04-11 11:15:00+00:00,0001-01-01 00:00:00+00:00,2023-04-21 15:51:48.524844+00:00,2023-04-20 13:41:10.964217+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:51:48.524844+00:00,00ucmrzqsbhjXJHmz2p7,8


<h2> Upsert Executions </h2>
Execution 1 was created a day before Execution 2. They both have the same quantity, but execution 1 has a price of 1560.00 whereas Execution 2 has a price of 1550.00.

In [23]:
execution_df = pd.read_csv("data/3/executions.csv")
display(execution_df)

,execution_id,placement_id,instrument_name,client_internal,isin,figi,quantity,state,side,type,price,currency,settlement_currency,settlement_currency_fx_rate,counterparty,created_date
0,EXEC001,PLC001,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,10,some_state,buy,limit,1550,USD,USD,1.0,some_counterparty,2023-04-12T11:30:00+00:00
1,EXEC002,PLC002,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,8,some_state,buy,limit,1560,USD,USD,1.0,some_counterparty,2023-04-12T11:45:00+00:00


In [24]:
set_request = helpers.create_execution_set_request_from_df(execution_df, scope)
response = api_factory.build(lusid.api.ExecutionsApi).upsert_executions(
    execution_set_request=set_request)

display(lusid_response_to_data_frame(response))

,id.scope,id.code,placement_id.scope,placement_id.code,properties,instrument_identifiers.Instrument/default/Figi,lusid_instrument_id,quantity,state,side,type,created_date,price.amount,price.currency,settlement_currency,settlement_currency_fx_rate,counterparty,version.effective_from,version.as_at_date,version.as_at_created,version.user_id_created,version.as_at_modified,version.user_id_modified,version.as_at_version_number
0,AllocationService_Example3,EXEC002,AllocationService_Example3,PLC002,{},BBG000BVPXP1,LUID_0000MIOX,8.0,some_state,buy,limit,2023-04-12 11:45:00+00:00,1560.0,USD,USD,1.0,some_counterparty,0001-01-01 00:00:00+00:00,2023-04-21 15:54:40.064514+00:00,2023-04-20 13:41:11.143282+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:54:40.064514+00:00,00ucmrzqsbhjXJHmz2p7,11
1,AllocationService_Example3,EXEC001,AllocationService_Example3,PLC001,{},BBG000BVPXP1,LUID_0000MIOX,10.0,some_state,buy,limit,2023-04-12 11:30:00+00:00,1550.0,USD,USD,1.0,some_counterparty,0001-01-01 00:00:00+00:00,2023-04-21 15:54:40.064514+00:00,2023-04-20 13:41:11.143282+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:54:40.064514+00:00,00ucmrzqsbhjXJHmz2p7,11


<h2> Upsert Allocations </h2>
These allocation simulate a previous run of the Allocation Service for Placement 1. You can see that Order 1 got alloated 2 shares whereas Order 2 got allocated 8 shares from execution 1. We will assume here that a user has overwritten the suggested shares from the Allocation Service. This will help us better demonstrate speific behaviours.

In [29]:
allocation_df = pd.read_csv("data/3/allocations.csv")
display(allocation_df)

,allocation_id,instrument_name,client_internal,isin,figi,quantity,price,currency,portfolio_id,order_id,execution_ids,placement_ids,date
0,ALLOC001,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,8,1550,USD,PORT001,ORD001,EXEC001,PLC001,2023-04-11T11:20:00+00:00
1,ALLOC002,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,2,1550,USD,PORT002,ORD002,EXEC001,PLC001,2023-04-11T11:15:00+00:00


In [30]:
set_request = helpers.create_allocation_set_request_from_df(allocation_df, scope)
response = api_factory.build(lusid.api.AllocationsApi).upsert_allocations(
    allocation_set_request=set_request)

display(lusid_response_to_data_frame(response))

,id.scope,id.code,allocated_order_id.scope,allocated_order_id.code,portfolio_id.scope,portfolio_id.code,quantity,instrument_identifiers.Instrument/default/Figi,version.effective_from,version.as_at_date,version.as_at_created,version.user_id_created,version.as_at_modified,version.user_id_modified,version.as_at_version_number,properties,instrument_scope,lusid_instrument_id,placement_ids.0.scope,placement_ids.0.code,state,side,type,date,price.amount,price.currency,execution_ids.0.scope,execution_ids.0.code
0,AllocationService_Example3,ALLOC002,AllocationService_Example3,ORD002,AllocationService_Example3,PORT002,2.0,BBG000BVPXP1,0001-01-01 00:00:00+00:00,2023-04-21 15:55:28.647171+00:00,2023-04-20 13:41:11.340905+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:55:28.647171+00:00,00ucmrzqsbhjXJHmz2p7,11,{},default,LUID_0000MIOX,AllocationService_Example3,PLC001,UnknownState,UnknownSide,UnknownType,2023-04-11 11:15:00+00:00,1550.0,USD,AllocationService_Example3,EXEC001
1,AllocationService_Example3,ALLOC001,AllocationService_Example3,ORD001,AllocationService_Example3,PORT001,8.0,BBG000BVPXP1,0001-01-01 00:00:00+00:00,2023-04-21 15:55:28.647171+00:00,2023-04-20 13:41:11.340905+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:55:28.647171+00:00,00ucmrzqsbhjXJHmz2p7,11,{},default,LUID_0000MIOX,AllocationService_Example3,PLC001,UnknownState,UnknownSide,UnknownType,2023-04-11 11:20:00+00:00,1550.0,USD,AllocationService_Example3,EXEC001


<h2> Run Allocation Service </h2>

In [31]:
result = api_factory.build(lusid.api.AllocationServiceApi).run_allocation_service([lusid.models.ResourceId(scope,"PLC002")])
display(result)

{'failed': {},
 'values': [{'code': 'PLC002', 'scope': 'AllocationService_Example3'}]}

<h2> Retrieve our Allocations </h2>
Here you can see that we have created two new allocations for Placement 2. You can see that the Allocation Service has taken into account the allocated shares from Placement one, with the generated Allocation for Order 1 having a quantity of 2, but a state of "Allocated". This ensures that across different placements for the same set of orders, no order will be over-allocated and allocations on the whole remain proportional.

In [32]:
allocations = api_factory.build(lusid.api.AllocationsApi).list_allocations(filter=f"allocatedOrderId.Scope eq \'{scope}\' and allocatedOrderId.Code in (\'ORD001', \'ORD002\')")
display(lusid_response_to_data_frame(allocations))

,id.scope,id.code,allocated_order_id.scope,allocated_order_id.code,portfolio_id.scope,portfolio_id.code,quantity,instrument_identifiers.Instrument/default/Figi,version.effective_from,version.as_at_date,properties,instrument_scope,lusid_instrument_id,placement_ids.0.scope,placement_ids.0.code,state,side,type,date,price.amount,price.currency,settlement_currency,execution_ids.0.scope,execution_ids.0.code,instrument_identifiers.Instrument/default/LusidInstrumentId
0,AllocationService_Example3,ALLOC002,AllocationService_Example3,ORD002,AllocationService_Example3,PORT002,2.0,BBG000BVPXP1,0001-01-01 00:00:00+00:00,2023-04-21 15:55:28.647171+00:00,{},default,LUID_0000MIOX,AllocationService_Example3,PLC001,UnknownState,UnknownSide,UnknownType,2023-04-11 11:15:00+00:00,1550.0,USD,None,AllocationService_Example3,EXEC001,NaN
1,AllocationService_Example3,ALLOC001,AllocationService_Example3,ORD001,AllocationService_Example3,PORT001,8.0,BBG000BVPXP1,0001-01-01 00:00:00+00:00,2023-04-21 15:55:28.647171+00:00,{},default,LUID_0000MIOX,AllocationService_Example3,PLC001,UnknownState,UnknownSide,UnknownType,2023-04-11 11:20:00+00:00,1550.0,USD,None,AllocationService_Example3,EXEC001,NaN
2,AllocationService_Example3,1173f3ed-e813-4b7a-95da-5d3b43cf7a8b,AllocationService_Example3,ORD002,AllocationService_Example3,PORT002,6.0,NaN,0001-01-01 00:00:00+00:00,2023-04-21 15:55:31.043155+00:00,{},default,LUID_0000MIOX,AllocationService_Example3,PLC002,PartiallyAllocated,buy,limit,2023-04-20 16:17:09.839744+00:00,1560.0,USD,USD,AllocationService_Example3,EXEC002,LUID_0000MIOX
3,AllocationService_Example3,ef98b08f-9a10-41a2-aac6-f36e3304a95b,AllocationService_Example3,ORD001,AllocationService_Example3,PORT001,2.0,NaN,0001-01-01 00:00:00+00:00,2023-04-21 15:55:31.043155+00:00,{},default,LUID_0000MIOX,AllocationService_Example3,PLC002,Allocated,buy,limit,2023-04-20 16:17:09.839719+00:00,1560.0,USD,USD,AllocationService_Example3,EXEC002,LUID_0000MIOX
